In [29]:
'''
import geopandas as gpd
from datetime import datetime, timezone, timedelta

grid_size = 0.0625
g5nr_frame_duration = timedelta(hours=1)
frame_duration = timedelta(minutes=10)
duration = timedelta(days=1)
start = datetime(year=2025, month=4, day=1, tzinfo=timezone.utc)

def get_clusters(frame):
    ds = dataset.sel(
        time=slice(
            (start+frame*g5nr_frame_duration).replace(day=7, month=1, year=2005, tzinfo=timezone.utc),
            (start+(frame+1)*g5nr_frame_duration).replace(day=7, month=1, year=2005, tzinfo=timezone.utc)
        ),
        lon=slice(-179, 179),
        lat=slice(-89,89)
    )

    #adding prospective features
    albedo = ds.isel(time=0)["albedo"].rio.set_spatial_dims("lon", "lat")
    preccon = ds.isel(time=0)["preccon"].rio.set_spatial_dims("lon", "lat")
    swgdn = ds.isel(time=0)["swgdn"].rio.set_spatial_dims("lon", "lat")
    prectot = ds.isel(time=0)["prectot"].rio.set_spatial_dims("lon", "lat")
    tauthgh = ds.isel(time=0)["tauthgh"].rio.set_spatial_dims("lon", "lat")
    tautmid = ds.isel(time=0)["tautmid"].rio.set_spatial_dims("lon", "lat")
    tautlow = ds.isel(time=0)["tautlow"].rio.set_spatial_dims("lon", "lat")
    tauttot = ds.isel(time=0)["tauttot"].rio.set_spatial_dims("lon", "lat")
    lwtup = ds.isel(time=0)["lwtup"].rio.set_spatial_dims("lon", "lat")
    precanv = ds.isel(time=0)["precanv"].rio.set_spatial_dims("lon", "lat")
    preclsc = ds.isel(time=0)["preclsc"].rio.set_spatial_dims("lon", "lat")

'''

'\nimport geopandas as gpd\nfrom datetime import datetime, timezone, timedelta\n\ngrid_size = 0.0625\ng5nr_frame_duration = timedelta(hours=1)\nframe_duration = timedelta(minutes=10)\nduration = timedelta(days=1)\nstart = datetime(year=2025, month=4, day=1, tzinfo=timezone.utc)\n\ndef get_clusters(frame):\n    ds = dataset.sel(\n        time=slice(\n            (start+frame*g5nr_frame_duration).replace(day=7, month=1, year=2005, tzinfo=timezone.utc),\n            (start+(frame+1)*g5nr_frame_duration).replace(day=7, month=1, year=2005, tzinfo=timezone.utc)\n        ),\n        lon=slice(-179, 179),\n        lat=slice(-89,89)\n    )\n\n    #adding prospective features\n    albedo = ds.isel(time=0)["albedo"].rio.set_spatial_dims("lon", "lat")\n    preccon = ds.isel(time=0)["preccon"].rio.set_spatial_dims("lon", "lat")\n    swgdn = ds.isel(time=0)["swgdn"].rio.set_spatial_dims("lon", "lat")\n    prectot = ds.isel(time=0)["prectot"].rio.set_spatial_dims("lon", "lat")\n    tauthgh = ds.isel(

In [30]:
'''import numpy as np
from netCDF4 import Dataset as netcdf
import scipy.ndimage as ndi
import datetime as dt

utc = dt.timezone.utc

def datetime_to_index(date):
    return int((dt.datetime.timestamp(date)-1116192600)/1800)

def lat_to_index(lat):
    return int((lat+90)/0.0625)

def lon_to_index(lon):
    return int((lon+180)/0.0625)


case = 'c1440_NR'  # for output file name
geosdir = 'https://opendap.nccs.nasa.gov/dods/OSSE/G5NR/Ganymed/7km/0.0625_deg/tavg/tavg30mn_2d_met3_Nx'
data = netcdf(geosdir, 'r')

outdir = '/Users/shashwatraj/Documents/Github/Code-Lab_RL_PriorityObs/Geos5datasets/'
tbthresh = 220   # brightness temperature threshold
sigma = 5.67037e-8

startdate = dt.datetime(2005,6,15, tzinfo = utc)  # initial date
enddate = dt.datetime(2007,6,15, tzinfo = utc)
step = dt.timedelta(minutes = 30)

    
#=== Domain Limits ===
lat1 = -90
lat2 = 90
lon1 = -179.8
lon2 = 179.8

lat_start = lat_to_index(lat1)
lat_stop = lat_to_index(lat2)
lon_start = lon_to_index(lon1)
lon_stop = lon_to_index(lon2)

lat = data.variables['lat'][:]
lon = data.variables['lon'][:]

# select within lat/lon limits
latind = np.logical_and( lat>=lat1, lat<=lat2 )
lonind = np.logical_and( lon>=lon1, lon<=lon2 )

lat = lat[latind]
lon = lon[lonind]
ny = lat.shape[0]
nx = lon.shape[0]

area = 0.*np.empty((ny,nx))
C = 2.*3.14159*6371  # earth circumference [km]                                    
d2r=3.14159/180 # degrees to radians 
lat2d = np.copy(area)
lon2d = np.copy(area)
dlon = lon[2]-lon[1]
for i in range(0,ny):
   area[i,:] = (C*np.cos(lat[i]*d2r)*dlon/360.)*(C*dlon/360.)
   lat2d[i,:] = lat[i]
for i in range(0,nx):
   lon2d[:,i] = lon[i]

date = startdate

print('Starting Loops')
# Loop through time
while date<enddate:
    
    datestr = str(date.year)+str(date.month).zfill(2)+str(date.day).zfill(2)
    datestr = datestr + "_"+str(date.hour).zfill(2)+str(date.minute).zfill(2)
    
    time_index = datetime_to_index(date)
    
    date = date + step
    
    #======= Load TB ========
    # Estimate from OLR assuming cloudtops radiate as blackbody

    olr = data.variables['lwtup'][time_index,lat_start:lat_stop,lon_start:lon_stop]
    tb = np.sqrt(np.sqrt(olr/sigma))   # stefan-boltzmann equation for effective temperature from radiation

    prectot = data.variables['prectot'][time_index,lat_start:lat_stop,lon_start:lon_stop]
    preccon = data.variables['preccon'][time_index,lat_start:lat_stop,lon_start:lon_stop]
    precanv = data.variables['precanv'][time_index,lat_start:lat_stop,lon_start:lon_stop]
    preclsc = data.variables['preclsc'][time_index,lat_start:lat_stop,lon_start:lon_stop]
    albedo = data.variables['albedo'][time_index,lat_start:lat_stop,lon_start:lon_stop]
    swgdn = data.variables['swgdn'][time_index,lat_start:lat_stop,lon_start:lon_stop]
    tauhgh = data.variables['tauhgh'][time_index,lat_start:lat_stop,lon_start:lon_stop]
    taumid = data.variables['taumid'][time_index,lat_start:lat_stop,lon_start:lon_stop]
    taulow = data.variables['taulow'][time_index,lat_start:lat_stop,lon_start:lon_stop]
    tautot = data.variables['tautot'][time_index,lat_start:lat_stop,lon_start:lon_stop]

    #=== define cloud mask as Tb less than 220 K ===
    
    cloudy = tb<tbthresh

    #============ Cluster statistics ===============

    labels, cnum = ndi.label(cloudy)  #labels features in an array/ "Labels" determines the number of clusters

    clstr_area = ndi.sum(area,labels,index=np.arange(1,cnum+1))
    clstr_preccon = ndi.mean(preccon,labels,index=np.arange(1,cnum+1))
    clstr_albedo = ndi.mean(albedo,labels,index=np.arange(1,cnum+1))
    clstr_swgdn = ndi.mean(swgdn,labels,index=np.arange(1,cnum+1))
    clstr_tauhgh = ndi.mean(tauhgh,labels,index=np.arange(1,cnum+1))
    clstr_taumid = ndi.mean(taumid,labels,index=np.arange(1,cnum+1))
    clstr_taulow = ndi.mean(taulow,labels,index=np.arange(1,cnum+1))
    clstr_tautot = ndi.mean(tautot,labels,index=np.arange(1,cnum+1))

    clstr_lat = ndi.mean(lat2d,labels,index=np.arange(1,cnum+1))
    clstr_lon = ndi.mean(lon2d,labels,index=np.arange(1,cnum+1))
    clstr_tbmin = ndi.minimum(tb,labels,index=np.arange(1,cnum+1))


    print("cluster count: "+str(cnum))


    #===== Save as netcdf =====

    outname = 'cluster_stats_'+case+'_tb'+str(tbthresh)+'K_'+datestr+'.nc4'
    print(" Saving: "+outdir+outname)
    ncwrite_id = netcdf( outdir+outname, 'w', format='NETCDF4' )

    ncwrite_id.createDimension( 'time', None )
    ncwrite_id.createDimension( 'cluster', cnum )

    clusterid = ncwrite_id.createVariable( 'cluster','f4', ('time','cluster',))
    sizesid  = ncwrite_id.createVariable('area','f4', ('time','cluster',) )
    tbminid  = ncwrite_id.createVariable('tbmin','f4', ('time','cluster',) )
    latid    = ncwrite_id.createVariable('lat','f4', ('time','cluster',) )
    lonid    = ncwrite_id.createVariable('lon','f4', ('time','cluster',) )
    albedoid = ncwrite_id.createVariable('albedo','f4', ('time','cluster',) )
    swgdnid  = ncwrite_id.createVariable('swgdn','f4', ('time','cluster',) )
    precconid = ncwrite_id.createVariable('preccon','f4', ('time','cluster',) )
    tauhghid = ncwrite_id.createVariable('tauhgh','f4', ('time','cluster',) )
    taumidid = ncwrite_id.createVariable('taumid','f4', ('time','cluster',) )
    taulowid = ncwrite_id.createVariable('taulow','f4', ('time','cluster',) )
    tautotid = ncwrite_id.createVariable('tautot','f4', ('time','cluster',) )
    


    clusterid[0,:] = np.arange(cnum)
    tbminid[0,:]   = clstr_tbmin[:cnum]
    sizesid[0,:]   = clstr_area[:cnum]
    precconid[0,:]  = clstr_preccon[:cnum]
    albedoid[0,:]  = clstr_albedo[:cnum]
    swgdnid[0,:]   = clstr_swgdn[:cnum]
    tauhghid[0,:] = clstr_tauhgh[:cnum]
    taumidid[0,:] = clstr_taumid[:cnum]
    taulowid[0,:] = clstr_taulow[:cnum]
    tautotid[0,:] = clstr_tautot[:cnum]
    latid[0,:]     = clstr_lat[:cnum]
    lonid[0,:]     = clstr_lon[:cnum]

    ncwrite_id.close()
    print('loop complete')'''

'import numpy as np\nfrom netCDF4 import Dataset as netcdf\nimport scipy.ndimage as ndi\nimport datetime as dt\n\nutc = dt.timezone.utc\n\ndef datetime_to_index(date):\n    return int((dt.datetime.timestamp(date)-1116192600)/1800)\n\ndef lat_to_index(lat):\n    return int((lat+90)/0.0625)\n\ndef lon_to_index(lon):\n    return int((lon+180)/0.0625)\n\n\ncase = \'c1440_NR\'  # for output file name\ngeosdir = \'https://opendap.nccs.nasa.gov/dods/OSSE/G5NR/Ganymed/7km/0.0625_deg/tavg/tavg30mn_2d_met3_Nx\'\ndata = netcdf(geosdir, \'r\')\n\noutdir = \'/Users/shashwatraj/Documents/Github/Code-Lab_RL_PriorityObs/Geos5datasets/\'\ntbthresh = 220   # brightness temperature threshold\nsigma = 5.67037e-8\n\nstartdate = dt.datetime(2005,6,15, tzinfo = utc)  # initial date\nenddate = dt.datetime(2007,6,15, tzinfo = utc)\nstep = dt.timedelta(minutes = 30)\n\n    \n#=== Domain Limits ===\nlat1 = -90\nlat2 = 90\nlon1 = -179.8\nlon2 = 179.8\n\nlat_start = lat_to_index(lat1)\nlat_stop = lat_to_index(lat2

In [31]:
import geojson
import glob
import os

input_dir = '/Users/shashwatraj/Documents/Github/Code-Lab_RL_PriorityObs/Geos5datasets/'
output_dir = 'Geos5datasets.geojson'


features=[]

for file in (glob.glob(os.path.join(input_dir,"*.nc4"))):
    timestamp = file.split('_')[-1].replace('.nc4', '')

    with netcdf(file,'r') as nc:
        tbmin = nc.variables['tbmin'][:]
        areas = nc.variables['area'][:]
        lat = nc.variables['lat'][:]
        lon = nc.variables['lon'][:]
        albedo = nc.variables['albedo'][:]
        swgdn = nc.variables['swgdn'][:]
        preccon = nc.variables['preccon'][:]
        precanv = nc.variables['precanv'][:]
        preclsc = nc.variables['preclsc'][:]
        tauthgh = nc.variables['tauthgh'][:]
        tautmid = nc.variables['tautmid'][:]
        tautlow = nc.variables['tautlow'][:]
        tauttot = nc.variables['tauttot'][:]
        prectot = nc.variables['prectot'][:]

        point = geojson.point(float(lon), float(lat))

        properties = {
            'timestamp': timestamp,
            'area': float(areas),
            'tbmin': float(tbmin),
            'albedo': float(albedo),
            'swgdn': float(swgdn),
            'preccon': float(preccon),
            'precanv': float(precanv),
            'preclsc': float(preclsc),
            'tauthgh': float(tauthgh),
            'tautmid': float(tautmid),
            'tautlow': float(tautlow),
            'tauttot': float(tauttot),
            'prectot': float(prectot)
        }

        feature = geojson.Feature(geometry=point, properties=properties)
        features.append(feature)

feature_collection = geojson.FeatureCollection(features)

with open(output_dir, 'w') as f:
    geojson.dump(feature_collection, f)

print(f"GeoJSON file created: {output_dir}")


GeoJSON file created: Geos5datasets.geojson


In [2]:
from tatc import utils
from tatc.schemas import PointedInstrument, Satellite, Instrument, TwoLineElements  

earthcare = Satellite(
    name = "EarthCare",
    orbit = TwoLineElements(
        tle=[
            "1 59908U 24101A   25200.34125573  .00010433  00000+0  14571-3 0  9999",
            "2 59908  97.0168 326.4971 0001222 108.6708 251.4681 15.57041891 64775"
        ]
    ),
    instruments=[
        Instrument(
            name = "CPR",
            field_of_regard=utils.swath_width_to_field_of_regard(394e3,650),
            cross_track_field_of_view = utils.swath_width_to_field_of_regard(394e3, 650),
            along_track_field_of_view = utils.swath_width_to_field_of_regard(394e3,1000)
        ),
        Instrument(
            name = "MSI",
            field_of_regard=utils.swath_width_to_field_of_regard(394e3,150e3) + 2*5.760868, #cross_track_field_of_view + 2*roll angle
            cross_track_field_of_view = utils.swath_width_to_field_of_regard(394e3, 150e3),
            along_track_field_of_view = utils.swath_width_to_field_of_regard(394e3, 500),
            roll_angle = 5.760868,  # degrees
            is_rectangular = True  
        )
    ]
)
        
satellites = [earthcare]

In [3]:
from datetime import datetime, timezone, timedelta
from tatc.analysis import compute_ground_track
import pandas as pd
from joblib import Parallel, delayed

startdate = datetime(2025,7,19,15,4, tzinfo = timezone.utc)  # initial date, discard the year. 
duration = timedelta(hours = 6)
step = timedelta(seconds = 10)
batch_duration = timedelta(minutes=10)

def compute_instrument_ground_track(satellite, start, duration, batch_duration, time_step):
    return pd.concat(
    Parallel(-1)(
        delayed(compute_ground_track)(
            satellite,
            pd.date_range(start + i*batch_duration, start + (i+1)*batch_duration, freq=time_step, inclusive="left"),
            crs="spice"
        ) 
        for i in range( duration // batch_duration)
    ),
    ignore_index=True
)



In [4]:
from copy import deepcopy

earthcare_cpr = deepcopy(earthcare)
earthcare_cpr.instruments = [inst for inst in earthcare.instruments if inst.name == "CPR"]

earthcare_msi = deepcopy(earthcare)
earthcare_msi.instruments = [inst for inst in earthcare.instruments if inst.name == "MSI"]

ground_tracks_cpr = compute_instrument_ground_track(
    earthcare_cpr, startdate, duration, batch_duration, step
)

ground_tracks_msi = compute_instrument_ground_track(
    earthcare_msi, startdate, duration, batch_duration, step
)


In [5]:
print(ground_tracks_cpr.head())
print(ground_tracks_cpr.columns)


                                            geometry  \
0  MULTIPOLYGON Z (((-22.38859 -23.47114 0, -22.3...   
1  MULTIPOLYGON Z (((-34.9835 -61.709 0, -34.9843...   
2  MULTIPOLYGON Z (((-114.99624 -83.0214 0, -114....   
3  MULTIPOLYGON Z (((-174.05683 -76.37688 0, -174...   
4  MULTIPOLYGON Z (((153.12835 -13.45434 0, 153.1...   

                       time  satellite instrument  valid_obs  
0 2025-07-19 15:04:00+00:00  EarthCare        CPR       True  
1 2025-07-19 15:14:00+00:00  EarthCare        CPR       True  
2 2025-07-19 15:24:00+00:00  EarthCare        CPR       True  
3 2025-07-19 15:34:00+00:00  EarthCare        CPR       True  
4 2025-07-19 15:44:00+00:00  EarthCare        CPR       True  
Index(['geometry', 'time', 'satellite', 'instrument', 'valid_obs'], dtype='object')


In [6]:
print(ground_tracks_msi.head())
print(ground_tracks_msi.columns)


                                            geometry  \
0  POLYGON Z ((-21.96067 -24.48583 0, -22.19311 -...   
1  POLYGON Z ((-34.32802 -62.78896 0, -34.81747 -...   
2  POLYGON Z ((-123.67586 -84.04609 0, -124.51569...   
3  MULTIPOLYGON Z (((-180 -76.04979 0, -180 -76.0...   
4  POLYGON Z ((151.90002 -12.79148 0, 151.91443 -...   

                       time  satellite instrument  valid_obs  
0 2025-07-19 15:04:00+00:00  EarthCare        MSI       True  
1 2025-07-19 15:14:00+00:00  EarthCare        MSI       True  
2 2025-07-19 15:24:00+00:00  EarthCare        MSI       True  
3 2025-07-19 15:34:00+00:00  EarthCare        MSI       True  
4 2025-07-19 15:44:00+00:00  EarthCare        MSI       True  
Index(['geometry', 'time', 'satellite', 'instrument', 'valid_obs'], dtype='object')


In [7]:
import matplotlib.pyplot as plt
import matplotlib.animation as animation
from cartopy import crs as ccrs
from IPython.display import HTML
from matplotlib.patches import Patch

fig, ax = plt.subplots(subplot_kw={"projection": ccrs.PlateCarree()})

frame_duration = batch_duration

def animate(frame):
    ax.clear()
    time = startdate + frame*frame_duration
    tracks_cpr = ground_tracks_cpr[
        (time <= ground_tracks_cpr.time) 
        & (ground_tracks_cpr.time < time + frame_duration)
    ]
    tracks_msi = ground_tracks_msi[
        (time <= ground_tracks_msi.time) 
        & (ground_tracks_msi.time < time + frame_duration)
    ]
    if not tracks_cpr.empty:
        tracks_cpr.plot(ax=ax, color="b", transform=ccrs.PlateCarree())
    if not tracks_msi.empty:
        tracks_msi.plot(ax=ax, color="r", transform=ccrs.PlateCarree())
    ax.set_global()
    ax.set_aspect("equal")
    ax.coastlines()
    ax.set_title(time)
    fig.tight_layout()

ani = animation.FuncAnimation(
    fig,
    animate, 
    frames=duration // frame_duration, 
    interval=100, 
    blit=False
)
display(HTML(ani.to_jshtml()))
plt.close()

In [8]:
grid_size = 0.0625
g5nr_frame_duration = timedelta(hours=1)

In [9]:
import rioxarray
import xarray as xr

dataset = xr.open_dataset(
    "https://opendap.nccs.nasa.gov/dods/OSSE/G5NR/Ganymed/7km/0.0625_deg/tavg/tavg30mn_2d_met3_Nx",
    decode_times=True,
)
#xr.open_dataset('https://opendap.nccs.nasa.gov/dods/OSSE/G5NR/Ganymed/7km/0.0625_deg/tavg/tavg30mn_2d_met3_Nx').to_netcdf('dataset.nc')
#dataset = xr.open_dataset('dataset.nc', decode_times=True)
dataset.rio.write_crs("epsg:4326", inplace=True)
dataset.rio.set_spatial_dims("lon", "lat", inplace=True)


/Users/shashwatraj/Documents/GitHub/Code-Lab_RL_PriorityObs/tatcenv/lib/python3.13/site-packages/xarray/coding/times.py:213: SerializationWarning: Ambiguous reference date string: 1-1-1 00:00:0.0. The first value is assumed to be the year hence will be padded with zeros to remove the ambiguity (the padded reference date string is: 0001-1-1 00:00:0.0). To remove this message, remove the ambiguity by padding your reference date strings with zeros.
  ref_date = _ensure_padded_year(ref_date)


<xarray.Dataset> Size: 138TB
Dimensions:      (time: 36576, lat: 2881, lon: 5760)
Coordinates:
  * time         (time) datetime64[ns] 293kB 2005-05-15T21:15:00 ... 2007-06-...
  * lat          (lat) float64 23kB -90.0 -89.94 -89.88 ... 89.88 89.94 90.0
  * lon          (lon) float64 46kB -180.0 -179.9 -179.9 ... 179.8 179.9 179.9
    spatial_ref  int64 8B 0
Data variables: (12/57)
    swgdnclr     (time, lat, lon) float32 2TB ...
    lwgabclrcln  (time, lat, lon) float32 2TB ...
    precsno      (time, lat, lon) float32 2TB ...
    ttauss       (time, lat, lon) float32 2TB ...
    prevtot      (time, lat, lon) float32 2TB ...
    cldhgh       (time, lat, lon) float32 2TB ...
    ...           ...
    tauhgh       (time, lat, lon) float32 2TB ...
    swtdn        (time, lat, lon) float32 2TB ...
    tsalt        (time, lat, lon) float32 2TB ...
    swgdn        (time, lat, lon) float32 2TB ...
    cldmid       (time, lat, lon) float32 2TB ...
    mxdiam       (time, lat, lon) float32 2TB ...
Attributes:
    title:                2d,30-Minute,Time-Averaged,Single-Level,Full Resolu...
    Conventions:          COARDS\nGrADS
    dataType:             Grid
    history:              Sun Jul 28 15:49:53 UTC 2024 : imported by GrADS Da...
    extra_das_attribute:  This is an example of metadata added using a supple...

In [10]:
constant = xr.open_dataset(
    "http://opendap.nccs.nasa.gov:80/dods/OSSE/G5NR/Ganymed/7km/0.0625_deg/const/const_2d_asm_Nx",
    decode_times=True,
)

#xr.open_dataset('http://opendap.nccs.nasa.gov:80/dods/OSSE/G5NR/Ganymed/7km/0.0625_deg/const/const_2d_asm_Nx').to_netcdf('constant.nc')
#constant = xr.open_dataset('dataset.nc', decode_times=True)
constant.rio.write_crs("epsg:4326", inplace=True)
constant.rio.set_spatial_dims("lon", "lat", inplace=True)


/Users/shashwatraj/Documents/GitHub/Code-Lab_RL_PriorityObs/tatcenv/lib/python3.13/site-packages/xarray/coding/times.py:213: SerializationWarning: Ambiguous reference date string: 1-1-1 00:00:0.0. The first value is assumed to be the year hence will be padded with zeros to remove the ambiguity (the padded reference date string is: 0001-1-1 00:00:0.0). To remove this message, remove the ambiguity by padding your reference date strings with zeros.
  ref_date = _ensure_padded_year(ref_date)


<xarray.Dataset> Size: 465MB
Dimensions:      (time: 1, lat: 2881, lon: 5760)
Coordinates:
  * time         (time) datetime64[ns] 8B 2005-05-16
  * lat          (lat) float64 23kB -90.0 -89.94 -89.88 ... 89.88 89.94 90.0
  * lon          (lon) float64 46kB -180.0 -179.9 -179.9 ... 179.8 179.9 179.9
    spatial_ref  int64 8B 0
Data variables:
    area         (time, lat, lon) float32 66MB ...
    frlake       (time, lat, lon) float32 66MB ...
    frland       (time, lat, lon) float32 66MB ...
    frlandice    (time, lat, lon) float32 66MB ...
    frocean      (time, lat, lon) float32 66MB ...
    phis         (time, lat, lon) float32 66MB ...
    sgh          (time, lat, lon) float32 66MB ...
Attributes:
    title:                2d,constant,Time-Invariant,Single-Level,Full Resolu...
    Conventions:          COARDS\nGrADS
    dataType:             Grid
    history:              Wed Feb 19 21:05:22 UTC 2025 : imported by GrADS Da...
    extra_das_attribute:  This is an example of metadata added using a supple...

In [11]:
import scipy.ndimage as ndi
import numpy as np
import geopandas as gpd
from shapely.geometry import box

threshold = 220  # brightness temperature threshold in Kelvin

def get_clusters(frame):
    # filter the dataset by time interval and latitude 
    ds = dataset.sel(
        time=slice(
            (startdate+frame*g5nr_frame_duration).replace(day = 20, month=7, year=2006,tzinfo=None), 
            (startdate+(frame+1)*g5nr_frame_duration).replace(day = 20, month=1, year=2007,tzinfo=None)
        ),
        lat=slice(-75,75)
    )
    # filter the constants by latitude (+/- 75)
    cs = constant.sel(
        lat=slice(-75,75)
    )
    # get data sets from g5nr
    lwtup = ds.isel(time=0)["lwtup"].rio.set_spatial_dims("lon", "lat")
    prectot = ds.isel(time=0)["prectot"].rio.set_spatial_dims("lon", "lat")
    tautot = ds.isel(time=0)["tautot"].rio.set_spatial_dims("lon", "lat")

    # perform the clustering based on lwtup threshold
    tb = np.sqrt(np.sqrt(lwtup / 5.67037e-8))  # effective temperature from radiation
    labels, _ = ndi.label(tb<threshold) # colder than 220 K
    
    # build a dataframe with cells having positive cluster label
    cells = gpd.GeoDataFrame(
        {
            "count": 1,
            "cluster": labels[labels>0],
            "time": startdate + frame*g5nr_frame_duration,
            "prectot": [v for v in prectot.where(labels>0).values.flatten() if ~np.isnan(v)],
            "tautot": [v for v in tautot.where(labels>0).values.flatten() if ~np.isnan(v)],
            "area": [v for v in cs["area"].where(labels>0).values.flatten() if ~np.isnan(v)]
        }, 
        geometry=[
            box(cell.lon, cell.lat, cell.lon+grid_size, cell.lat+grid_size) 
            for row in prectot.where(labels>0) 
            for cell in row 
            if ~np.isnan(cell)
        ],
        crs="EPSG:4326"
    )
    # add other columns (cell count, cell area, and total prectot)
    cells["tot_prectot"] = cells["prectot"] * cells["area"]
    cells["avg_prectot"] = cells["prectot"]
    cells["max_prectot"] = cells["prectot"]

    cells["tot_tautot"] = cells["tautot"] * cells["area"]
    cells["avg_tautot"] = cells["tautot"]
    cells["max_tautot"] = cells["tautot"]
    
    return cells[cells.cluster>0].dissolve(
        by=["time", "cluster"], 
        aggfunc={
            "count": "sum", 
            "area": "sum", 
            "tot_prectot": "sum", 
            "avg_prectot": "mean", 
            "max_prectot": "max",
            "tot_tautot": "sum",
            "avg_tautot": "mean",
            "max_tautot": "max"
        }
    )

'''clusters = pd.concat(
    Parallel(n_jobs=-1)(
        delayed(get_clusters)(
            frame,
        )
        for frame in range(duration // g5nr_frame_duration)
    ),
).reset_index()
'''
results = []
for frame_df in range(duration // g5nr_frame_duration):
    try:
        clusters = get_clusters(frame_df)
        results.append(clusters)
    except Exception as e:
        print(f"Frame {frame_df} failed: {e}")
clusters = pd.concat(results).reset_index()

display(clusters)

Error:DAP DATADDS packet is apparently too short
syntax error, unexpected WORD_WORD, expecting SCAN_ATTR or SCAN_DATASET or SCAN_ERROR
context: <html^><body><h1>504 Gateway Time-out</h1>The server didn't respond in time.</body></html>
syntax error, unexpected WORD_WORD, expecting SCAN_ATTR or SCAN_DATASET or SCAN_ERROR
context: <html^><body><h1>504 Gateway Time-out</h1>The server didn't respond in time.</body></html>


: 

: 